In [1]:
import io
import pandas as pd
from pandas import json_normalize

import numpy as np
import json
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False,nb_workers=2)

from bs4 import BeautifulSoup
import gspread as gs
from datetime import datetime

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
cashbac_key='Cashbac-GCP-Keys.json'
horego_key='Horego-GCP-Keys.json'

In [3]:
import sys
import os

cur_directory=os.getcwd()
parent_directory=cur_directory[:cur_directory.rfind('/')]
sys.path.append(parent_directory)
 
# importing
import MyFunc
from MyFunc import *

from importlib import reload
reload(MyFunc)
from MyFunc import *

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [4]:
import google.auth
from google.oauth2 import service_account
from google.cloud import bigquery


credentials = service_account.Credentials.from_service_account_file('Cashbac-GCP-Keys.json')
project_id = 'cashbac-31433'

# Make clients.
cashbac_bqclient = bigquery.Client(
    credentials=credentials,
    project=project_id,
)

import google.auth
from google.oauth2 import service_account
from google.cloud import bigquery

credentials = service_account.Credentials.from_service_account_file('Horego-GCP-Keys.json')
project_id = 'horego-big-query'

# Make clients.
horego_bqclient = bigquery.Client(
    credentials=credentials,
    project=project_id,
)

import psycopg2.pool
def db_connect_cashbac_nopool(host,database,user,password):
    """
    :return:
    """
    connection = psycopg2.connect(host=host,
                              database=database,
                              user=user,
                              password=password,
                              port=5432)
    return connection

In [5]:
cashbac_key='Cashbac-GCP-Keys.json'
horego_key='Horego-GCP-Keys.json'

name_exclude=get_name_exclude(horego_key)
subtype_exclude=get_subtype_exclude(horego_key)
exclude_restorant=get_exclude_restorant(horego_key)
include_restorant=get_include_restaurant_type1(horego_key)
outlet_in_mall_type1=get_include_restaurant_mall_type1(horego_key)

name_exclude.KEYWORD=name_exclude.KEYWORD.str.replace("'",'')
subtype_exclude.KEYWORD=subtype_exclude.KEYWORD.str.replace("'",'')

In [6]:
subtype_condition=''
for subtype in subtype_exclude.KEYWORD.unique():
    subtype_condition=subtype_condition+' '+f"""and lower(subtypes) not like lower('%{subtype}%')"""

name_condition=''
for name in name_exclude.KEYWORD.unique():
    name_condition=name_condition+' '+f"""and lower(name) not like lower('%{name}%')"""

exclude_restorant_condition=''
for place_id in exclude_restorant.place_id.unique():
    exclude_restorant_condition=exclude_restorant_condition+' '+f"""and lower(place_id) != lower('{place_id}')"""

In [7]:
include_restorant_condition='lower(place_id) in ('
for place_id in include_restorant[(include_restorant.place_id!='')].place_id.unique():
    place_id=place_id.lower()
    include_restorant_condition=include_restorant_condition+' '+f"""'{place_id}',"""
include_restorant_condition=include_restorant_condition[:-1]+')'


include_restorant_condition_google_id='lower(google_id) in ('
for google_id in include_restorant[(include_restorant.google_id!='')].google_id.unique():
    google_id=google_id.lower()
    include_restorant_condition_google_id=include_restorant_condition_google_id+' '+f"""'{place_id}',"""
include_restorant_condition_google_id=include_restorant_condition_google_id[:-1]+')'

In [8]:
include_restorant_condition2='lower(place_id) in ('
for place_id in outlet_in_mall_type1[(outlet_in_mall_type1.place_id!='')].place_id.unique():
    place_id=place_id.lower()
    include_restorant_condition2=include_restorant_condition2+' '+f"""'{place_id}',"""
include_restorant_condition2=include_restorant_condition2[:-1]+')'


include_restorant_condition_google_id2='lower(google_id) in ('
for google_id in outlet_in_mall_type1[(outlet_in_mall_type1.google_id!='')].google_id.unique():
    google_id=google_id.lower()
    include_restorant_condition_google_id2=include_restorant_condition_google_id2+' '+f"""'{google_id}',"""
include_restorant_condition_google_id2=include_restorant_condition_google_id2[:-1]+')'

In [9]:
%%time
#######################################################################################

query_string=f""" 

select * from 
(
    SELECT * 
    FROM `cashbac-31433.cashbac_datalake_prod.vw_outcrapper_google_results_raw` table1
    where true 
    and 
    (
        {include_restorant_condition}
    )
    and not exists
    (
      select 1 
      from `cashbac-31433.cashbac_datalake_prod.vw_outcrapper_google_results` table2
      where table1.google_id=table2.google_id
    )
)table1
union all
select * from 
(
    SELECT * 
    FROM `cashbac-31433.cashbac_datalake_prod.vw_outcrapper_google_results_raw` table1
    where true 
    and 
    (
        {include_restorant_condition_google_id}
    )
    and not exists
    (
      select 1 
      from `cashbac-31433.cashbac_datalake_prod.vw_outcrapper_google_results` table2
      where table1.google_id=table2.google_id
    )
)table12
union all
select * from 
(
    SELECT * 
    FROM `cashbac-31433.cashbac_datalake_prod.vw_outcrapper_google_results_raw` table1
    where true 
    and 
    (
        {include_restorant_condition2}
    )
    and not exists
    (
      select 1 
      from `cashbac-31433.cashbac_datalake_prod.vw_outcrapper_google_results` table2
      where table1.google_id=table2.google_id
    )
)table3
union all
select * from 
(
    SELECT * 
    FROM `cashbac-31433.cashbac_datalake_prod.vw_outcrapper_google_results_raw` table1
    where true 
    and 
    (
        {include_restorant_condition_google_id2}
    )
    and not exists
    (
      select 1 
      from `cashbac-31433.cashbac_datalake_prod.vw_outcrapper_google_results` table2
      where table1.google_id=table2.google_id
    )
)table13
"""

query_result = list(cashbac_bqclient.query(query_string).result(timeout=None))
print(len(query_result))
if len(query_result)>0:
    data = pd.DataFrame(data=[list(x.values()) for x in query_result], columns=list(query_result[0].keys()))
    data=data.drop_duplicates()
    print(len(data))

del query_result

0
CPU times: user 99.3 ms, sys: 35.8 ms, total: 135 ms
Wall time: 16.6 s


In [64]:
query_string=f""" 

SELECT * 
FROM `cashbac-31433.cashbac_datalake_prod.outcrapper_google_results` table1
limit 1

"""

query_result = list(cashbac_bqclient.query(query_string).result(timeout=None))
print(len(query_result))
if len(query_result)>0:
    df_temp = pd.DataFrame(data=[list(x.values()) for x in query_result], columns=list(query_result[0].keys()))

1


In [65]:
query_string# data=pd.read_csv('results/transfer_from_raw.csv',sep=';')
# data=cleaning_data_merchants(data)
# hot_area=get_hot_area_list(horego_key)

# data['hot_area']=data.apply(lambda x:get_hot_area(x.latitude,x.longitude,hot_area),axis=1)
data_location=pd.read_csv('daftar_kota_admin_level_kelurahan_with_lat_long.csv',sep=';')
data_location=data_location[(data_location.latitude<=6) &(data_location.latitude>=-11)&(data_location.longitude>=95)&(data_location.longitude<=141)]

# data=filtering_data_merchants(data,'Cashbac-GCP-Keys.json')
print('Total : ',len(data))
if len(data)>0:
    data[['kodepos','province','kabupaten','kecamatan','kelurahan']]=data.parallel_apply(lambda x: get_post_code(x.longitude,x.latitude,data_location),axis=1)
    
    # df_temp=pd.read_csv('results/clear_raw_table_filtered.csv',sep=';', nrows=1)
    my_schema=[]
    save_columns=[]
    for column in df_temp.columns :
        if column in data.columns :
            save_columns.append(column)
            if column in ['working_hours','about']:
                my_schema.append(bigquery.SchemaField(column, "JSON"))
            else:
                my_schema.append(bigquery.SchemaField(column, "STRING"))

    data[save_columns].to_csv('results/temp_upload_filtered.csv',sep=';',index=False)
    
    #############################################################################################
    #BQ DATALAKE HOREGO
    src_path_file='results/temp_upload_filtered.csv'
    target_path_file=f"scrapping_results/temp_upload_filtered.csv"
    bucket_name='horego-bq'
    upload_file_to_GCS(horego_key,bucket_name,src_path_file,target_path_file)
    
    table_schema = "horego_datalake_dev"
    table_name = "outcrapper_google_results"
    gcs_path_uri = f"gs://{bucket_name}/scrapping_results/temp_upload_filtered.csv"
    create_table_from_gcs(horego_key,gcs_path_uri,table_schema,table_name,my_schema,'append')
    
    #############################################################################################
    #############################################################################################
    #BQ DATALAKE CASHBAC
    bucket_name='cashbac_datalake'
    src_path_file='results/temp_upload_filtered.csv'
    target_path_file=f"scrapping_results/temp_upload_filtered.csv"
    upload_file_to_GCS(cashbac_key,bucket_name,src_path_file,target_path_file)
    
    table_schema = "cashbac_datalake_prod"
    table_name = "outcrapper_google_results"
    gcs_path_uri = f"gs://{bucket_name}/scrapping_results/temp_upload_filtered.csv"
    create_table_from_gcs(cashbac_key,gcs_path_uri,table_schema,table_name,my_schema,'append')
    
    # table_schema = "public_datalake"
    # table_name = "outcrapper_google_results"
    # gcs_path_uri = f"gs://{bucket_name}/scrapping_results/temp_upload_filtered.csv"
    # create_table_from_gcs(cashbac_key,gcs_path_uri,table_schema,table_name,my_schema,'append')
    #############################################################################################
    print('')

print('DONE')

Total :  143
Uploading results/temp_upload_filtered.csv
Path in GCS: horego-bq/scrapping_results/temp_upload_filtered.csv
table on : horego_datalake_dev.outcrapper_google_results
Uploading results/temp_upload_filtered.csv
Path in GCS: cashbac_datalake/scrapping_results/temp_upload_filtered.csv
table on : cashbac_datalake_prod.outcrapper_google_results

DONE


In [66]:
%%time

query_string=f""" 

SELECT column_name
FROM cashbac-31433.cashbac_datalake_prod.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'list_insert_to_prod'

"""

query_result = list(cashbac_bqclient.query(query_string).result(timeout=None))
print(len(query_result))
df_temp = pd.DataFrame(data=[list(x.values()) for x in query_result], columns=list(query_result[0].keys()))

my_schema=[]
save_columns=[]
for column in df_temp.column_name :
    if column in data.columns :
        save_columns.append(column)
        my_schema.append(bigquery.SchemaField(column, "STRING"))

data[save_columns].drop_duplicates().to_csv('results/input_list_insert_to_prod.csv',sep=';',index=False)

5
CPU times: user 20.2 ms, sys: 10.5 ms, total: 30.7 ms
Wall time: 673 ms


In [67]:
cashbac_key='Cashbac-GCP-Keys.json'
horego_key='Horego-GCP-Keys.json'

#############################################################################################

# df_temp=pd.read_csv('results/choosen_photos.csv',sep=';', nrows=1)
# my_schema=[]
# for column in df_temp.columns :
#     my_schema.append(bigquery.SchemaField(column, "STRING"))

src_path_file='results/input_list_insert_to_prod.csv'
target_path_file='scrapping_results/input_list_insert_to_prod.csv'
bucket_name='cashbac_datalake'
upload_file_to_GCS(cashbac_key,bucket_name,src_path_file,target_path_file)

table_schema = "cashbac_datalake_prod"
table_name = "list_insert_to_prod"
gcs_path_uri = f"gs://{bucket_name}/scrapping_results/input_list_insert_to_prod.csv"
create_table_from_gcs(cashbac_key,gcs_path_uri,table_schema,table_name,my_schema,'append')

Uploading results/input_list_insert_to_prod.csv
Path in GCS: cashbac_datalake/scrapping_results/input_list_insert_to_prod.csv
table on : cashbac_datalake_prod.list_insert_to_prod
